# 最終課題

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin, urlparse
# 1. 初期設定
START_URL = "https://www.musashino-u.ac.jp/"
DOMAIN = urlparse(START_URL).netloc
SLEEP_TIME = 0.5  # Webサーバーへの負荷軽減のため、待機時間を設定（秒）
MAX_PAGES = 6000  # URLの上限
# サイトマップ格納用の辞書
sitemap = {}
# 既に処理済みのURLを記録し、無限ループや重複アクセスを防ぐためのセット
visited_urls = set()
# 処理待ちのURLのリスト（キューとして利用）
urls_to_visit = [START_URL]
print(f"--- スクレイピング開始: {START_URL} の同一ドメインを探索 ---")
# 静的ファイル拡張子の除外セット
EXCLUDE_EXTS = {
    '.pdf', '.png', '.jpg', '.jpeg', '.gif', '.svg', '.bmp', '.ico', '.webp'
}
# 2. メインのスクレイピングループ
while urls_to_visit and len(sitemap) < MAX_PAGES:
    current_url = urls_to_visit.pop(0)
    # 既に訪問済み、または同一ドメインでない場合はスキップ
    if current_url in visited_urls or urlparse(current_url).netloc != DOMAIN:
        continue
    print(f"アクセス中: {current_url}")
    visited_urls.add(current_url)
    # 3. 負荷軽減のための待機
    time.sleep(SLEEP_TIME)
    try:
        # 4. ページへのアクセス
        response = requests.get(current_url, timeout=5)
        response.raise_for_status()  # HTTPエラーがあれば例外を発生させる
        response.encoding = response.apparent_encoding  # 文字化け対策
        # 5. Beautiful Soupでパース（高速化のためlxmlが入っていれば推奨）
        soup = BeautifulSoup(response.text, 'html.parser')
        # 6. <title>タグの抽出と辞書への格納
        title_tag = soup.find('title')
        page_title = title_tag.string.strip() if title_tag and title_tag.string else "タイトルなし"
        sitemap[current_url] = page_title
        # 7. 同一ドメインの新しいリンクを抽出
        for link in soup.find_all('a', href=True):
            # 絶対URLに変換
            absolute_url = urljoin(current_url, link['href'])
            # クエリパラメータやフラグメントを削除してクリーンなURLにする
            clean_url = absolute_url.split('#')[0].split('?')[0]
            # 同一ドメイン以外はスキップ
            if urlparse(clean_url).netloc != DOMAIN:
                continue
            # 静的ファイル拡張子の除外
            path = urlparse(clean_url).path.lower()
            if any(path.endswith(ext) for ext in EXCLUDE_EXTS):
                continue
            # 未訪問かつ未登録であれば、リストに追加
            if clean_url not in visited_urls and clean_url not in urls_to_visit:
                urls_to_visit.append(clean_url)
    except requests.exceptions.RequestException as e:
        # アクセスエラーの処理
        print(f"エラー: {current_url} - {e}")
    except Exception as e:
        # その他エラーの処理
        print(f"処理エラー: {current_url} - {e}")
# 8. 最終結果の表示 (print()で表示する要件)
print("\n--- サイトマップ結果 ---")
for url, title in sitemap.items():
    print(f"URL: {url}")
    print(f"Title: {title}")
    print("-" * 20)
# 件数を表示
print(f"\n--- 取得件数 ---")
print(f"総URL数: {len(sitemap)}")

--- スクレイピング開始: https://www.musashino-u.ac.jp/ の同一ドメインを探索 ---
アクセス中: https://www.musashino-u.ac.jp/
アクセス中: https://www.musashino-u.ac.jp/access.html
アクセス中: https://www.musashino-u.ac.jp/admission/request.html
アクセス中: https://www.musashino-u.ac.jp/contact.html
アクセス中: https://www.musashino-u.ac.jp/prospective-students.html
アクセス中: https://www.musashino-u.ac.jp/students.html
アクセス中: https://www.musashino-u.ac.jp/alumni.html
アクセス中: https://www.musashino-u.ac.jp/parents.html
アクセス中: https://www.musashino-u.ac.jp/business.html
アクセス中: https://www.musashino-u.ac.jp/guide/
アクセス中: https://www.musashino-u.ac.jp/guide/profile/
アクセス中: https://www.musashino-u.ac.jp/guide/activities/
アクセス中: https://www.musashino-u.ac.jp/guide/campus/
アクセス中: https://www.musashino-u.ac.jp/guide/facility/
アクセス中: https://www.musashino-u.ac.jp/guide/information/
アクセス中: https://www.musashino-u.ac.jp/guide/profile/media/
アクセス中: https://www.musashino-u.ac.jp/admission/
アクセス中: https://www.musashino-u.ac.jp/admission/faculty/
アクセス中